In [ ]:
%pip install python-dotenv langchain langchain-openai openai

In [ ]:
%pip install -U langchain langchain-classic langchain-openai openai langchain-google-genai langchain-community langchain-tavily 

In [ ]:
%pip install -U langchain-google-genai langchain

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

openai_key = os.getenv("OPENAI_API_KEY")
gemini_key = os.getenv("GOOGLE_API_KEY")

print(openai_key[:5])
print(gemini_key[:5])


In [ ]:
from langchain_openai import ChatOpenAI 
from langchain.tools import tool 
from langchain.agents import create_agent
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse 

 
gpt_model = ChatOpenAI(model="gpt-4o-mini") 

gemini_model = ChatGoogleGenerativeAI(model="gemini-2.5-flash",
    max_retries=6, # Will wait and try again automatically
    temperature=0) 


@tool 
def search(query: str) -> str:
    """Search for information."""
    return f"Results for: {query}" 

@tool 
def get_weather(location: str) -> str:
    """Get current weather information for a given city."""
    return f"Weather in {location}: Sunny, 72°F" 

 
@wrap_model_call 
def dynamic_model_selection(request: ModelRequest, handler) -> ModelResponse:
    message_count = len(request.state["messages"]) 

    if message_count > 10: 
        # Use an advanced model for longer conversations 
        model = gpt_model
    else: 
        model = gemini_model 

    return handler(request.override(model=model)) 


agent = create_agent(model=gemini_model, # Default model 
    tools=[search, get_weather], 
    middleware=[dynamic_model_selection] 
)

response = agent.invoke(
    {"messages": [{"role": "user", "content": "location: Bangalore, what is the weather in Bangalore?"}]},
    context={"user_role": "expert"}
)

#response = agent.invoke("What is LangChain?")
print(f"Response: {response}")

print("===================")

for message in response["messages"]:
    print(message.content)


